## Adding missing informtion to JSON files.


[Format](https://en.wikipedia.org/wiki/GeoJSON)

In [11]:
import json
import glob
import os

import geocoder

In [18]:
# Enter the filename if you want to process only a single file.
for file in glob.glob('../data/output/text_ner/*.json'):
    with open (file, 'r') as f:
        json_file = json.load(f)
        for feature in json_file["features"]:
            feature_label = feature["properties"]["source_label"]
            # With GeoNames URL
            g = geocoder.geonames(feature_label, key='kartriert')
            feature["properties"]["url"] = "https://www.geonames.org/" + str(g.geonames_id)
            # Placeholder
            # feature["properties"]["url"] = "https://www.geonames.org/None"
            json_dump = json.dumps(json_file, indent=4)
            with open('../data/output/text_ner/with_url/' + os.path.basename(file) + '_test', 'w') as f_w:
                f_w.write(json_dump)

        # json_dump = json.dumps(json_file, indent=4)
        # with open('../data/output/text_ner/with_url/' + os.path.basename(file), 'w') as f_w:
        #     f_w.write(json_dump)


Error the hourly limit of 1000 credits for sarahondraszek has been exceeded. Please throttle your requests or use the commercial service. from JSON {'status': {'message': 'the hourly limit of 1000 credits for sarahondraszek has been exceeded. Please throttle your requests or use the commercial service.', 'value': 19}}
Error the hourly limit of 1000 credits for sarahondraszek has been exceeded. Please throttle your requests or use the commercial service. from JSON {'status': {'message': 'the hourly limit of 1000 credits for sarahondraszek has been exceeded. Please throttle your requests or use the commercial service.', 'value': 19}}
Error the hourly limit of 1000 credits for sarahondraszek has been exceeded. Please throttle your requests or use the commercial service. from JSON {'status': {'message': 'the hourly limit of 1000 credits for sarahondraszek has been exceeded. Please throttle your requests or use the commercial service.', 'value': 19}}
Error the hourly limit of 1000 credits f

KeyboardInterrupt: 